In [1]:
# Importando libraries

from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import pandas as pd
import selenium
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller
import requests
import os
import glob
import zipfile
import time
from pathlib import Path


In [2]:
# setup-selenium

chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

driver = webdriver.Chrome()
driver.maximize_window()

In [3]:
url = 'https://apps.fas.usda.gov/psdonline/app/index.html#/app/downloads'
driver.get(url)

In [4]:
dataset_xpath = '/html/body/div[2]/div[3]/div/div[2]/div/ul/li[3]/a/uib-tab-heading'
driver.find_element("xpath",dataset_xpath).click()

In [5]:
# all paths
xpath_dict = {
    
            'coffee'      : '/html/body/div[2]/div[3]/div/div[2]/div/div/div[3]/div/div[2]/datasets/div/div[2]/div/div/table/tbody/tr[3]/td[3]/a'  ,
            'cotton'     : '/html/body/div[2]/div[3]/div/div[2]/div/div/div[3]/div/div[2]/datasets/div/div[2]/div/div/table/tbody/tr[4]/td[3]/a'  ,
            'dairy'      : '/html/body/div[2]/div[3]/div/div[2]/div/div/div[3]/div/div[2]/datasets/div/div[2]/div/div/table/tbody/tr[5]/td[3]/a'  ,
            'grain'      : '/html/body/div[2]/div[3]/div/div[2]/div/div/div[3]/div/div[2]/datasets/div/div[2]/div/div/table/tbody/tr[8]/td[3]/a'  ,
            'livestock'  : '/html/body/div[2]/div[3]/div/div[2]/div/div/div[3]/div/div[2]/datasets/div/div[2]/div/div/table/tbody/tr[10]/td[3]/a' ,
            'oilseeds'   : '/html/body/div[2]/div[3]/div/div[2]/div/div/div[3]/div/div[2]/datasets/div/div[2]/div/div/table/tbody/tr[11]/td[3]/a' ,
            'russia_data': '/html/body/div[2]/div[3]/div/div[2]/div/div/div[3]/div/div[2]/datasets/div/div[2]/div/div/table/tbody/tr[12]/td[3]/a' ,
            'sugar'      : '/html/body/div[2]/div[3]/div/div[2]/div/div/div[3]/div/div[2]/datasets/div/div[2]/div/div/table/tbody/tr[13]/td[3]/a' ,
            
            }

In [6]:
def get_name(dir_path):
    
    with zipfile.ZipFile(dir_path, mode="r") as archive:
         for filename in archive.namelist():
                return filename


def get_file_dataframe():
    
    list_of_files = glob.glob(f'{Path.home()}/Downloads/*.zip') # * means all if need specific format then *.csv
    #print(list_of_files)
    latest_file = max(list_of_files, key=os.path.getctime)
    filename = get_name(latest_file)
    
    zf = zipfile.ZipFile(latest_file)
    df = pd.read_csv(zf.open(filename))
    
    return df

def delete_files_used(times):
    
    for i in range(times):
        
        list_of_files = glob.glob(f'{Path.home()}/Downloads/*.zip') # * means all if need specific format then *.csv
        latest_file = max(list_of_files, key=os.path.getctime)
        os.remove(latest_file)
        #print('ok')
        
    return 

def download_wait(path_to_downloads):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < 20:
        time.sleep(1)
        dl_wait = False
        for fname in os.listdir(path_to_downloads):
            if fname.endswith('.crdownload'):
                dl_wait = True
        seconds += 1
    return seconds

In [7]:
data_dict = {}

for key, values in xpath_dict.items():
    
    driver.find_element("xpath",values).click()
    download_wait(f'{Path.home()}/Downloads/')
    
    df = get_file_dataframe()
    data_dict[key] = df.copy()
    
    

In [8]:
driver.close()

In [9]:
n_times = len(xpath_dict)

delete_files_used(n_times);

In [10]:
for values in data_dict.values():
    
    values.Attribute_Description = values.Attribute_Description.apply(lambda x: x.lower().replace(' ','-').replace("''","").replace("/","").replace(" ","-"))
    values.Commodity_Description = values.Commodity_Description.apply(lambda x: x.lower().replace(', ','-').replace("''","").replace("/","").replace(" ","-"))
    values.Country_Code = values.Country_Code.apply(lambda x : str(x).lower().replace("''","").replace("/","").replace(" ","-"))
    values.Unit_Description = values.Unit_Description.apply(lambda x: str(x).lower().replace("''","").replace("/","").replace(" ","-"))
    
    

In [11]:
base_path = 'C:\\Users\\guilh\\TM\\var_comodity\\JSONS'

In [12]:
each_loop_time = {}

In [13]:
#Transformar tudo em JSON! 


for keys,values in data_dict.items():
    print(keys)
    start = time.time()
    
    custom_path = f"\\USDA\\{keys}\\"     
    path = base_path + custom_path
    
    for commodity in values.Commodity_Description.unique():
        comm_filter = (values.Commodity_Description == commodity)
        
        for code in values.Country_Code.unique():
            country_filter = (values.Country_Code == code)
            
            for attribute in values.Attribute_Description.unique():
                attr_filter = (values.Attribute_Description == attribute)
                
                filter_df = comm_filter & country_filter & attr_filter
                df_filter = values[filter_df]
                
                unit = str(df_filter.Unit_Description.unique()).replace('[',"").replace("]","").replace("'","")
                
                name = commodity+'-'+attribute+'_'+code+'_'+'und'+'_y1.json'
                name = name.replace('(',"").replace(")","")
                complete_path = path+name
    
                df_to_json = df_filter[['Market_Year','Value']]
                #df_to_json.to_json(complete_path,indent=1)
                df = df_to_json
                if not df.empty:
            #    print(df.empty)
                    df.to_json(complete_path,indent =1)
                else:
                    pass
    
    stop = time.time()
    duration = (stop - start)/60
    print(duration)
    each_loop_time[keys] = duration
    

coffee
0.1669391671816508
cotton
0.23155717849731444
dairy
0.5807786186536154
grain
11.760514756043753
livestock
2.157295258839925
oilseeds
51.87662766377131
russia_data
0.00028477907180786134
sugar
0.5997870842615763


In [14]:
tempo_p_rodar = pd.DataFrame(data = each_loop_time, index=['tempo']).T.tempo.sum()

In [15]:
#tempo_p_rodar